<a href="https://colab.research.google.com/github/Roh15/Analysis-of-Lok-Sabha-Dataset/blob/main/Calculate_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import regex as re

In [ ]:
def countFiles():
  file_count = dict.fromkeys(list(range(1,18)), 0)
  for ls in range(1,18):
    files = 0
    for file in range(1,1000):
      filepath = '/content/drive/MyDrive/Lok Sabha Data/textfiles/' + str(ls) + '/' + str(file) + '.csv'
      if os.path.isfile(filepath):
        files += 1
    file_count[ls] = files
  return file_count

In [ ]:
num_files = countFiles()

In [ ]:
def remainingNERfiles(ls):
  not_done = []
  for file in range(1,num_files[ls]):
    filepath = '/content/drive/MyDrive/Lok Sabha Data/NER/' + str(ls) + '/' + str(file) + '_NER.csv'
    if not os.path.isfile(filepath):
      not_done.append(file)
  print('LS-' + str(ls) + ' remainig NER files are: ' + str(not_done))

In [ ]:
def remainingPOSfiles(ls):
  not_done = []
  for file in range(1,num_files[ls]):
    filepath = '/content/drive/MyDrive/Lok Sabha Data/POS/' + str(ls) + '/' + str(file) + '_POS.csv'
    if not os.path.isfile(filepath):
      not_done.append(file)
  print('LS-' + str(ls) + ' remainig POS files are: ' + str(not_done))

In [ ]:
for i in range(16,17):
  remainingNERfiles(i)

In [ ]:
remainingPOSfiles(16)

In [ ]:
entities = [
  'PERSON',
  'NORP',
  'FAC',
  'ORG',
  'GPE',
  'LOC',
  'PRODUCT',
  'EVENT',
  'WORK_OF_ART',
  'LAW',
  'LANGUAGE',
  'DATE',
  'TIME',
  'PERCENT',
  'MONEY',
  'QUANTITY',
  'ORDINAL',
  'CARDINAL']

In [ ]:
ls_entity_counts = dict.fromkeys(list(range(1,18)))

In [ ]:
def countEntities(ls):
  ls_entities = dict.fromkeys(entities, 0)
  for file in range(1,num_files[ls]):
    filepath = '/content/drive/MyDrive/Lok Sabha Data/NER/' + str(ls) + '/' + str(file) + '_NER.csv'
    try:
      nerfile = pd.read_csv(filepath, usecols=['entity_type'],lineterminator='\n')
      entity_list = nerfile['entity_type'].values
      for entity in entity_list:
        ls_entities[entity] += 1
    except FileNotFoundError as e:
      continue
  ls_entity_counts[ls] = ls_entities

In [ ]:
for i in [8,15,16,17]:
  countEntities(i)

In [ ]:
tags = [
  'NOUN',
  'PROPN']

In [ ]:
def countPOSTags(ls):
  ls_tags = dict.fromkeys(tags, 0)
  for file in range(1,num_files[ls]):
    filepath = '/content/drive/MyDrive/Lok Sabha Data/POS/' + str(ls) + '/' + str(file) + '_POS.csv'
    try:
      posfile = pd.read_csv(filepath, usecols=['word_upos'],lineterminator='\n')
      tag_list = posfile['word_upos'].values
      for tag in tag_list:
        try:
          ls_tags[tag] += 1
        except KeyError as e:
          continue
    except FileNotFoundError as e:
      continue
  print(ls_tags)

In [ ]:
countPOSTags(16)

In [ ]:
ls = 14
file = 1
filepath = '/content/drive/MyDrive/Lok Sabha Data/NER/' + str(ls) + '/' + str(file) + '_NER.csv'
nerfile = pd.read_csv(filepath, usecols=['entity_text', 'entity_type'])
nerfile.head()

In [ ]:
def maxFrequency(my_list):
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
  
    freq_max = dict(sorted(freq.items(), key=lambda item: item[1], reverse=True))
    count = 0
    for key, value in freq_max.items():
        print ("% s : % d"%(key, value))
        count+=1
        if count == 3:
          break

In [ ]:
for entity in entities:
  instances = []
  print(' ')
  print(entity)
  for i in range(len(nerfile)):
    if nerfile.loc[i]['entity_type'] == entity:
      instances.append(nerfile.loc[i]['entity_text'])
  maxFrequency(instances)

In [ ]:
# make a stat file listing all entities frequencies

entity_counter = []
ls = 16
already_encountered = []
for file in tqdm(range(1,
                       #num_files[ls]
                       332)):
  filepath = '/content/drive/MyDrive/Lok Sabha Data/NER/' + str(ls) + '/' + str(file) + '_NER.csv'
  try:
    ner_file = pd.read_csv(filepath, lineterminator='\n')
    for index, row in ner_file.iterrows():
      try:
        match = re.search('\p{Devanagari}', row['entity_text'])
      except:
        continue
      if match == None:
        try:
          i = already_encountered.index(row['entity_text'])
          entity_counter[i][2] += 1
        except ValueError as e:
          entity_counter.append([row['entity_text'], row['entity_type'], 1])
          already_encountered.append(row['entity_text'])
  except FileNotFoundError as e:
    print(str(file) + 'not found')
    continue

statfilepath = '/content/drive/MyDrive/Lok Sabha Data/LS16_NER_Stats.csv'
pd.DataFrame(entity_counter, columns=['entity_text', 'entity_type', 'instances']).to_csv(statfilepath)

In [ ]:
# make a stat file listing all noun frequencies

noun_counter = []
ls = 16
already_encountered = []
for file in tqdm(range(1,
                       #num_files[ls]
                       332)):
  filepath = '/content/drive/MyDrive/Lok Sabha Data/Hindi+English POS-NER/' + str(ls) + '/' + str(file) + '_POS-NER.csv'
  try:
    pos_file = pd.read_csv(filepath, lineterminator='\n')
    for index, row in pos_file.iterrows():
      if row['word_upos'] == 'NOUN' or row['word_upos'] == 'PROPN':
        try:
          i = already_encountered.index(row['word/entity'])
          noun_counter[i][3] += 1
        except ValueError as e:
          noun_counter.append([row['word/entity'], row['word_upos'], row['word_xpos'], 1])
          already_encountered.append(row['word/entity'])
  except FileNotFoundError as e:
      continue

statfilepath = '/content/drive/MyDrive/Lok Sabha Data/LS16_Noun_Stats.csv'
pd.DataFrame(noun_counter, columns=['word', 'upos', 'xpos', 'instances']).to_csv(statfilepath)

In [ ]:
# filter prop nouns

arr = []
filepath = '/content/drive/MyDrive/Lok Sabha Data/LS16_Noun_Stats.csv'
target_file = pd.read_csv(filepath, lineterminator='\n')
for index, row in target_file.iterrows():
  if row['upos'] == 'NOUN':
    try:
      match = re.search('\p{Devanagari}', row['word'])
    except:
      continue
    if match == None:
      continue
    else:
      arr.append([row['word'], row['instances']])

statfilepath = '/content/drive/MyDrive/Lok Sabha Data/LS16_PropN.csv'
pd.DataFrame(arr, columns=['word', 'instances']).to_csv(statfilepath)